# STAMMP Experimental Processing, Fordyce Lab
## Standard Template
___

In [2]:
import sys

sys.path.insert(0, '/Users/Arjun/ProcessingPack/processingpack-stammp/')

In [3]:
import chip
import experiment as exp
import chipcollections as collections
import skimage
import numpy as np

import pandas as pd
import matplotlib.pyplot as pl
%matplotlib inline
%config InlineBackend.figure_format = 'svg'


In [4]:
d = {'x':[],'y':[],'step':[],'id':[]}
indexing_columns = pd.DataFrame(data=d);


#create a small function that appends a unique index to all columns not considered an "indexing"
#column

def append_suffix(indexing_columns,df,suffix):
    #indexing columns contains all the columns which have names which should NOT be changed
    #df is where you want to append the suffix to the columns
    #suffix, string of what you want to append
    
    result = df.copy();
    
    result.columns = np.where(result.columns.isin(indexing_columns)==False, result.columns +'_' + suffix, df.columns);
    return result;

## 1. Establish Experiment and Pinlist

In [5]:
#insert descriptors of experiment
#root: folder containing archive of experiment in storage source
#description: description of experiment (date, protein, oligo)
#operator: initials of person who ran experiment

root = 'directory to experiment here'
description = 'date_protein_oligo'
operator = 'initials'

e = exp.Experiment(description, root, operator)

INFO     Experiment Initialized | Description: 190708_PHO4_CGCGTG, Operator: AA


In [6]:
#identify pinlist directory and pull in for analysis
pinlist_path = 'directory to pinlist file'

pinlist = e.read_pinlist(pinlist_path)
pinlist.head(5)

Indices MutantID
x y                 
1 1  (1, 1)    Empty
  2  (1, 2)    Empty
  3  (1, 3)    Empty
  4  (1, 4)    Empty
  5  (1, 5)    Empty

In [7]:
#sanity check for whole pinlist size--should be 1568 rows long (number of chambers in device)
pinlist.shape

(1568, 2)

## 2. Add Devices and Corners

In [8]:
#add corners of the device--these corners mark the rough center positions of the corner button positions
d1_corners = ((547,472), (5930,436), (575,5949), (5967,5916))
d1 = exp.Device('s4', 'd1', (28, 56), pinlist, d1_corners)

In [9]:
e.addDevices([d1])

INFO     Added Device | Device: FordyceLab, s4, d1


## 3. Process Standard(s)

In [10]:
# Create a standard series, load stitched images. Done with prewash DNA images in Cy5 channel

p_d1 = 'directory to PrewashCy5 images folder'
d1_cMU = collections.StandardSeries(d1, 'Note about experiment')

#channel ('5cy5') below and exposure time (50 ms below) might vary based on experiment 
d1_cMU.load_files(p_d1, '5cy5', 50)

In [11]:
d1_cMU.process()
d1_cMU.save_summary_images()
d1_cMU.save_summary()

## S2. Process Reference Quantification(s)

##### Load

In [12]:
#load starting postwash GFP image to identify all spots of proteins in device

#p= directory to sample image
p = 'directory to sample GFP image'
d1_GFPQuant = collections.ChipQuant(d1, 'ButtonQuant')

d1_GFPQuant.load_file(p, '4egfp', 500)

##### Process

In [13]:
d1_GFPQuant.process()
quant_report = d1_GFPQuant.summarize()
d1_GFPQuant.save_summary_image()

## Process postwash Images

In [14]:
#define new chip series for each postwash GFP and postwash Cy5
cs_postwash_GFP = collections.ChipSeries(d1, 'postwash_images', 'step')

cs_postwash_Cy5 = collections.ChipSeries(d1, 'postwash_images', 'step')


In [15]:
#load in root to postwash GFP images
root = 'directory to folder of postwash GFP (TF) images'
cs_postwash_GFP.load_files(root, '4egfp', 500)

#load in root to postwash Cy5 images
root = 'directory to folder of postwash Cy5 (DNA) images'
cs_postwash_Cy5.load_files(root, '5Cy5', 3000)

In [16]:
#use the GFP reference image to quantify all GFP and all Cy5 images

#quant GFP
cs_postwash_GFP.map_from(d1_GFPQuant.chip, mapto_args = {'features': 'button'})
cs_postwash_GFP_df = cs_postwash_GFP.summarize()
#save CSV to destination directory/filename
cs_postwash_GFP_df.to_csv('')

cs_postwash_GFP_df = cs_postwash_GFP.save_summary_images(featuretype = 'button')

#quant Cy5
cs_postwash_Cy5.map_from(d1_GFPQuant.chip, mapto_args = {'features': 'button'})
cs_postwash_Cy5_df = cs_postwash_Cy5.summarize()
#save CSV to destination directory/filename
cs_postwash_Cy5_df.to_csv('')

cs_postwash_Cy5_df = cs_postwash_Cy5.save_summary_images(featuretype = 'button')